#Anchors for Heart Disease dataset:

In [1]:
!pip install alibi

     |████████████████████████████████| 337kB 28.8MB/s 
     |████████████████████████████████| 2.5MB 36.4MB/s 
     |████████████████████████████████| 28.0MB 108kB/s 
     |████████████████████████████████| 3.3MB 27.3MB/s 
     |████████████████████████████████| 901kB 24.3MB/s 
  Created wheel for spacy-lookups-data: filename=spacy_lookups_data-0.1.0-py2.py3-none-any.whl size=28052158 sha256=644797f06cb4da0eb9235d822787c0e4c118006162fe5a56153eee36656976e2
  Stored in directory: /root/.cache/pip/wheels/2a/2b/0a/d6fb6235c56d014d224bca760d15d7cbdd820813085ffcd35d
Successfully built spacy-lookups-data


In [2]:
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

TF version:  2.5.0
Eager execution enabled:  False


In [3]:
df = pd.read_csv('/content/healthcare-dataset-stroke-data.csv')
df.head(5)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
df.shape

(5110, 12)

In [9]:
feature_names=df.columns.to_list()
data = df[feature_names]
target = df.stroke

Splitting the data into training and testing set

In [10]:
x_train, x_test, y_train, y_test = train_test_split(data,target, random_state=0)

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from sklearn import svm

In [15]:
clf = svm.SVC(kernel='linear', C = 1.0,probability=True)

In [17]:
svc_model = clf.fit(x_train,y_train)

ValueError: ignored

In [11]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(C=0.01,solver='liblinear',max_iter=500)
model.fit(x_train,y_train)

ValueError: ignored

Apllying AnchorTabular method 

In [ ]:
from alibi.explainers import AnchorTabular
predict_func=lambda x:model.predict_proba(x)
explainer=AnchorTabular(predict_func,feature_names)

In [ ]:
model.predict_proba(x_test)

array([[0.58247803, 0.41752197],
       [0.77450519, 0.22549481],
       [0.75681729, 0.24318271],
       [0.18091057, 0.81908943],
       [0.86033967, 0.13966033],
       [0.11171929, 0.88828071],
       [0.40723555, 0.59276445],
       [0.19783263, 0.80216737],
       [0.11024544, 0.88975456],
       [0.18950502, 0.81049498],
       [0.56097183, 0.43902817],
       [0.91997135, 0.08002865],
       [0.64081597, 0.35918403],
       [0.47946202, 0.52053798],
       [0.92404139, 0.07595861],
       [0.19096939, 0.80903061],
       [0.78200054, 0.21799946],
       [0.41555401, 0.58444599],
       [0.20363026, 0.79636974],
       [0.53557304, 0.46442696],
       [0.86416571, 0.13583429],
       [0.60047315, 0.39952685],
       [0.19753243, 0.80246757],
       [0.19349917, 0.80650083],
       [0.81447944, 0.18552056],
       [0.77601218, 0.22398782],
       [0.60509221, 0.39490779],
       [0.27504511, 0.72495489],
       [0.54251071, 0.45748929],
       [0.62958067, 0.37041933],
       [0.

In [ ]:
x_train=x_train.to_numpy()
x_test=x_test.to_numpy()

In [ ]:
explainer.fit(x_train)

AnchorTabular(meta={
    'name': 'AnchorTabular',
    'type': ['blackbox'],
    'explanations': ['local'],
    'params': {'seed': None, 'disc_perc': (25, 50, 75)}
})

In [ ]:
from sklearn.metrics import jaccard_similarity_score
yjc = model.predict(x_test)
jaccard_similarity_score(y_test, yjc)

jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.


0.76

Taking an instance where the person has no disease

In [ ]:
inst=1
target_label=['no heart disease','heart disease']
print('Person has',target_label[explainer.predictor(x_test[inst].reshape(1, -1))[0]])
anchor=explainer.explain(x_test[inst])
print('Anchor generated feature(/s)',anchor.anchor)

Person has no heart disease
Anchor generated feature(/s) ['thalach > 138.00', 'thal <= 0.00', 'age <= 48.00']


Here, we can see that as the person's maximum heart rate is 152 (which is greater than 138) and thal value is 0 'normal' and age is less than 48, the above features act as anchors for the patient and deduce that the person has no heart disease 

Taking another instance where the person has a heart disease

In [ ]:
print('Person has',target_label[explainer.predictor(x_test[9].reshape(1, -1))[0]])
anchor=explainer.explain(x_test[9])
print('Anchor generated feature(/s)',anchor.anchor)

Person has heart disease
Anchor generated feature(/s) ['thalach <= 138.00', 'ca > 1.00']


Here, we can see that as the person's maximum heart rate is 131 (which is less than 138) and blood vessels coloured by fluoroscopy are 3 (greater than 1), the above features act as anchors for the patient and deduce that the person has a heart disease 